<a href="https://colab.research.google.com/github/mkielo3/mammalian_brains/blob/main/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q numpy pandas scipy scikit-learn matplotlib seaborn pillow ipykernel
!pip install -q torch torchvision torchaudio
!pip install -q altair datasets
!pip install -q numba numbasom==0.0.5
!pip install -q tensorboard


In [5]:
!git clone https://github.com/mkielo3/mammalian_brains.git
%cd mammalian_brains

Cloning into 'mammalian_brains'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 253 (delta 6), reused 19 (delta 4), pack-reused 222 (from 1)
Receiving objects: 100% (253/253), 97.68 MiB | 33.19 MiB/s, done.
Resolving deltas: 100% (71/71), done.
/content/mammalian_brains


In [6]:
import torch
import pandas as pd

from tqdm import tqdm
from numbasom.core import lattice_closest_vectors
from analysis.som import SOM
from utils import save_output_to_pickle

from models.olfaction.olfaction import Olfaction
from models.vision.vision import Vision
from models.audio.audio import Audio
from models.touch.touch import Touch
from models.memory.memory import Memory

from config import Args

In [ ]:
    args = Args()
    args.setup_models = False # Set to True if first ever time running
    args.experiment_name = "main_results"

    for modality in ([Olfaction(args), Vision(args), Audio(args), Touch(args), Memory(args)]):
        print ("\n", modality.modality, pd.Timestamp.now())

        # 1. Train/Download Model
        modality.setup_model()
        modality.setup_som()

        # 2. Get activations for each patch
        patches = modality.get_patches()
        activation_list = []
        for p in tqdm(patches):
            p, static = modality.generate_static(p)
            activation = modality.calculate_activations(static)
            activation_list.append([p, activation])

        # 3. Fit SOM
        x_mat = torch.stack([x[1] for x in activation_list]).numpy()
        som = modality.initialize_som(SOM)
        lattice = som.train(x_mat, num_iterations=args.som_epochs, initialize=args.som_init, normalize=False, start_lrate=args.som_lr)

        # 4. Get coordinates for each BMU
        coordinate_list = [x[0] for x in activation_list]
        closest = lattice_closest_vectors(x_mat, lattice, additional_list=coordinate_list)

        # 5. Save
        output = {"closest": closest,
                "coord_map": coordinate_list,
                "x_range": (0, max([x[0][0] for x in activation_list])),
                "y_range": (0, max([x[0][1] for x in activation_list])),
                "lattice": lattice,
                "som": None,
                "samples": modality.sample_data,
                "modality": modality.modality,
                "args": args,
                "activations": activation_list
                }

        save_output_to_pickle(output, args.experiment_name)

/content/mammalian_brains/models/olfaction/olfaction.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{self.project_path}/models/olfac


 olfaction 2025-02-07 06:32:14.350133


100%|██████████| 4096/4096 [00:02<00:00, 1463.59it/s]


Initializing SOM with Random
Done Init
SOM training took: 14.742145 seconds.
Finding closest data points took: 0.544560 seconds.
Saved output to: output/main_results/olfaction_20250207_063232.pkl

 vision 2025-02-07 06:32:32.882206


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
 48%|████▊     | 21.5M/44.7M [00:00<00:00, 104MB/s]

In [ ]:
import torch

imgs = torch.load(f"models/vision/saved_data/processed_images.pt")

torch.Size([100, 3, 224, 224])

In [ ]:
imgs_subset = imgs[:50].clone().contiguous()
torch.save(imgs_subset, "models/vision/saved_data/processed_images.pt")

In [ ]:

imgs_subset.shape

torch.Size([50, 3, 224, 224])

In [ ]:
data = torch.load(f"models/olfaction/saved_data/val_dataset.pt", map_location='cpu')

In [ ]:
imgs_subset = data.clone().contiguous()
torch.save(imgs_subset, "models/olfaction/saved_data/val_dataset.pt")


In [ ]:
!ls -l

total 3280
-rw-r--r--  1 matthewkielo  staff     2745 Feb  7 00:53 README.md
drwxr-xr-x  6 matthewkielo  staff      192 Feb  7 00:53 analysis
-rw-r--r--  1 matthewkielo  staff     2110 Feb  7 00:57 config.py
-rw-r--r--  1 matthewkielo  staff  1639511 Feb  7 01:02 foobar.pt
-rw-r--r--  1 matthewkielo  staff     2157 Feb  7 00:53 main.py
drwxr-xr-x  9 matthewkielo  staff      288 Feb  7 00:53 models
-rw-r--r--  1 matthewkielo  staff     4260 Feb  7 00:57 quickstart.ipynb
-rw-r--r--  1 matthewkielo  staff     7420 Feb  7 00:53 requirements.txt
-rw-r--r--  1 matthewkielo  staff      689 Feb  7 00:53 setup.py
-rw-r--r--  1 matthewkielo  staff      507 Feb  7 00:53 utils.py


In [ ]:
1639511
163841200